In [1]:
# 모듈 로딩
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.feature_extraction.text import CountVectorizer
from konlpy.tag import Kkma,Okt
from datetime import timedelta
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import datetime

In [502]:
# 새로운 stock df 만들기
stock_df = pd.read_csv("./project_data/YG.csv")
stock_df=stock_df[["날짜","종가"]]
stock_df["날짜"] = pd.to_datetime(stock_df["날짜"])
stock_df 

# 등락 만들기
stock_df1=stock_df["종가"]
stock_df2=stock_df["종가"]
stock_df2 = stock_df2.iloc[1:]
stock_df2=stock_df2.reset_index(drop=True)

list1= []
for i in range(len(stock_df2)):
    list1.append((stock_df1.iloc[i] - stock_df2.iloc[i])/stock_df2.iloc[i]*100)

stock_df=stock_df.iloc[:-1]

stock_df["등락"] = list1
stock_df

stock_df["등락수"]=stock_df["등락"].apply(lambda x : 1 if x > 2 else 0)
# stock_df["등락수"]=stock_df["등락수"].astype(int)

In [503]:
stock_df=stock_df[stock_df["날짜"]>="2017-06-08"]

In [504]:
stock_df["날짜"] = pd.to_datetime(stock_df["날짜"]).dt.date

In [505]:
# 뉴스 감성분석

news_df = pd.read_csv("./project_data/2023_news_senti_transformer.csv")

news_df["뉴스감성분석"] = news_df["0"].apply(lambda x : 1 if x =="positive" else 0 if x =="neutral" else -1)

news_df=news_df.reset_index(drop=True)

news_df["날짜"]=pd.to_datetime(news_df["날짜"]).dt.date

news_df=news_df.groupby("날짜").mean("뉴스감성분석")

news_df=news_df.reset_index()
news_df["날짜"]=pd.to_datetime(news_df["날짜"]).dt.date
news_df["날짜"] =news_df["날짜"] +timedelta(days=1)

In [506]:
# 토론 감성분석

debate_df = pd.read_csv("./project_data/종목토론_감성분석_2017_2023.csv")

debate_df["토론감성분석"] = debate_df["0"].apply(lambda x : 1 if x =="positive" else 0 if x =="neutral" else -1)

debate_df=debate_df.reset_index(drop=True)

debate_df["날짜"]=pd.to_datetime(debate_df["날짜"]).dt.date

debate_df=debate_df.groupby("날짜").mean("토론감성분석")

debate_df=debate_df.reset_index()
debate_df["날짜"]=pd.to_datetime(debate_df["날짜"]).dt.date
debate_df["날짜"] =debate_df["날짜"] +timedelta(days=1)

In [507]:
stock_df=stock_df.merge(debate_df,how="inner",on="날짜").merge(news_df,how="inner",on="날짜")

In [508]:
stock_df

,날짜,종가,등락,등락수,1,토론감성분석,뉴스감성분석
0,2023-10-04,60000.0,-4.458599,0,0.915191,-0.072727,-0.3
1,2023-09-27,62800.0,-2.786378,0,0.941830,-0.051903,-0.2
2,2023-09-26,64600.0,-1.374046,0,0.942046,-0.062057,0.1
3,2023-09-25,65500.0,-1.355422,0,0.942519,-0.019868,0.1
4,2023-09-22,66400.0,-4.046243,0,0.932153,-0.018223,-0.1
...,...,...,...,...,...,...,...
180,2023-01-06,49600.0,1.018330,0,0.946236,-0.037037,0.1
181,2023-01-05,49100.0,0.614754,0,0.977178,0.185185,0.2
182,2023-01-04,48800.0,-1.414141,0,0.922232,0.027778,0.4
183,2023-01-03,49500.0,3.017690,1,0.950920,-0.025000,0.1


In [509]:
trainx,testx,trainy,testy = train_test_split(stock_df[["토론감성분석"]],stock_df["등락수"],random_state=10,stratify=stock_df["등락수"])

In [510]:
# 랜덤포레스트
rc = RandomForestClassifier()
rc.fit(trainx,trainy)
print(rc.score(trainx,trainy))
print(rc.score(testx,testy))
pred= rc.predict(testx)
print(classification_report(testy,pred))

0.9347826086956522
0.6808510638297872
              precision    recall  f1-score   support

           0       0.79      0.77      0.78        35
           1       0.38      0.42      0.40        12

    accuracy                           0.68        47
   macro avg       0.59      0.59      0.59        47
weighted avg       0.69      0.68      0.68        47



In [511]:
#KNN
knn = KNeighborsClassifier()
knn.fit(trainx,trainy)
print(knn.score(trainx,trainy))
print(knn.score(testx,testy))
pred= knn.predict(testx)
print(classification_report(testy,pred))

0.8188405797101449
0.6808510638297872
              precision    recall  f1-score   support

           0       0.76      0.83      0.79        35
           1       0.33      0.25      0.29        12

    accuracy                           0.68        47
   macro avg       0.55      0.54      0.54        47
weighted avg       0.65      0.68      0.66        47



In [512]:
#XGBOOST
xg = XGBClassifier()
xg.fit(trainx,trainy)
print(xg.score(trainx,trainy))
print(xg.score(testx,testy))
pred= xg.predict(testx)
print(classification_report(testy,pred))

0.9202898550724637
0.6170212765957447
              precision    recall  f1-score   support

           0       0.77      0.69      0.73        35
           1       0.31      0.42      0.36        12

    accuracy                           0.62        47
   macro avg       0.54      0.55      0.54        47
weighted avg       0.66      0.62      0.63        47



c:\Users\KDP-024\anaconda3\envs\deep2\lib\site-packages\xgboost\data.py:335: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
c:\Users\KDP-024\anaconda3\envs\deep2\lib\site-packages\xgboost\data.py:338: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  is_categorical_dtype(dtype) or is_pa_ext_categorical_dtype(dtype)
c:\Users\KDP-024\anaconda3\envs\deep2\lib\site-packages\xgboost\data.py:384: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype):
c:\Users\KDP-024\anaconda3\envs\deep2\lib\site-packages\xgboost\data.py:359: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  return is_int or 

In [513]:
from keras.models import Sequential
from keras.layers import Dense

In [514]:
# DNN
model =Sequential()
model.add(Dense(10, input_shape=(1,),activation="relu"))
model.add(Dense(100,activation="relu"))
model.add(Dense(100,activation="relu"))
model.add(Dense(10,activation="relu"))

model.add(Dense(2,activation="softmax"))

model.compile(optimizer="adam",loss="sparse_categorical_crossentropy",metrics=["accuracy"])
model.fit(x=trainx,y=trainy,epochs=200,validation_split=0.2)

Epoch 1/200
4/4 [==============================] - 1s 42ms/step - loss: 0.6907 - accuracy: 0.7455 - val_loss: 0.6817 - val_accuracy: 0.7500
Epoch 2/200
4/4 [==============================] - 0s 10ms/step - loss: 0.6800 - accuracy: 0.7455 - val_loss: 0.6730 - val_accuracy: 0.7500
Epoch 3/200
4/4 [==============================] - 0s 11ms/step - loss: 0.6701 - accuracy: 0.7455 - val_loss: 0.6609 - val_accuracy: 0.7500
Epoch 4/200
4/4 [==============================] - 0s 11ms/step - loss: 0.6577 - accuracy: 0.7455 - val_loss: 0.6462 - val_accuracy: 0.7500
Epoch 5/200
4/4 [==============================] - 0s 12ms/step - loss: 0.6418 - accuracy: 0.7455 - val_loss: 0.6290 - val_accuracy: 0.7500
Epoch 6/200
4/4 [==============================] - 0s 11ms/step - loss: 0.6249 - accuracy: 0.7455 - val_loss: 0.6095 - val_accuracy: 0.7500
Epoch 7/200
4/4 [==============================] - 0s 13ms/step - loss: 0.6068 - accuracy: 0.7455 - val_loss: 0.5895 - val_accuracy: 0.7500
Epoch 8/200
4/4 [===

In [515]:
model.predict(testx)

2/2 [==============================] - 0s 2ms/step


array([[0.8048998 , 0.19510019],
       [0.7177256 , 0.28227443],
       [0.8153738 , 0.18462618],
       [0.7332694 , 0.2667306 ],
       [0.8075588 , 0.1924412 ],
       [0.81460005, 0.18539998],
       [0.8087866 , 0.1912134 ],
       [0.7960952 , 0.20390484],
       [0.79348505, 0.20651501],
       [0.7762398 , 0.22376014],
       [0.6546114 , 0.34538865],
       [0.7762398 , 0.22376014],
       [0.6780099 , 0.3219901 ],
       [0.7063499 , 0.29365012],
       [0.80430573, 0.19569427],
       [0.7368701 , 0.26312992],
       [0.7762398 , 0.22376014],
       [0.61535895, 0.38464108],
       [0.7762398 , 0.22376014],
       [0.62656933, 0.37343064],
       [0.7762398 , 0.22376014],
       [0.7395726 , 0.2604274 ],
       [0.81353843, 0.1864616 ],
       [0.80742157, 0.19257843],
       [0.7133061 , 0.28669384],
       [0.7762398 , 0.22376014],
       [0.55176693, 0.44823304],
       [0.7762398 , 0.22376014],
       [0.8147886 , 0.1852114 ],
       [0.8158329 , 0.18416704],
       [0.

In [516]:
pred=model.predict(testx)

2/2 [==============================] - 0s 3ms/step


In [517]:
preds= []
for i in pred:
    preds.append(i.argmax())
preds = np.array(preds)

In [518]:
print(classification_report(testy,preds))

              precision    recall  f1-score   support

           0       0.74      1.00      0.85        35
           1       0.00      0.00      0.00        12

    accuracy                           0.74        47
   macro avg       0.37      0.50      0.43        47
weighted avg       0.55      0.74      0.64        47



c:\Users\KDP-024\anaconda3\envs\deep2\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\KDP-024\anaconda3\envs\deep2\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\KDP-024\anaconda3\envs\deep2\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [531]:
debate_df

,날짜,1,토론감성분석
0,2017-06-08,0.935142,-0.098765
1,2017-06-09,0.956819,-0.066667
2,2017-06-10,0.941544,-0.023810
3,2017-06-11,0.901544,0.083333
4,2017-06-12,0.944238,0.090909
...,...,...,...
2285,2023-10-01,0.937898,-0.100000
2286,2023-10-02,0.925228,-0.009804
2287,2023-10-03,0.914511,0.031250
2288,2023-10-04,0.915191,-0.072727
